# 12.11 & 13.11. RAG Chatbot 📚

📍 [Download notebook](https://github.com/maxschmaltz/Course-LLM-based-Assistants/tree/main/llm-based-assistants/sessions/block2_core/1211_1311/1211_1311.ipynb)

In today'l lab, we will be expanding the chatbot we created in our [previous session](../0611.ipynb). We'll implement a RAG functionality so that the chatbot has access to custom knowledge. In the first part (12.11), we'll preprocess our data for further retrieval. Next day (13.11), we will complete the RAG chatbot and use the data we have preprocessed in the first part to inject our custom knowledge to the LLM. 

Our plan for 12.11 & 13.11:

* [Data Preprocessing](#data)
* [Prompting al LangChain](#prompts)
* [Simple RAG](#rag)
* [Advanced RAG](#adv_rag)

## Prerequisites

To start with the tutorial, complete the steps [Prerequisites](../../../../infos/llm_inference_guide/README.md#prerequisites), [Environment Setup](../../../../infos/llm_inference_guide/README.md#environment-setup), and [Getting API Key](../../../../infos/llm_inference_guide/README.md#getting-api-key) from the [LLM Inference Guide](../../../../infos/llm_inference_guide/README.md).

Today, we have more packages so we'll use the requirements file to install the dependencies:

In [ ]:
!curl -o requirements.txt https://raw.githubusercontent.com/maxschmaltz/Course-LLM-based-Assistants/main/llm-based-assistants/sessions/block2_core/1211_1311/requirements.txt
!pip install -r requirements.txt

Finally, download the data we'll be working with:

In [ ]:
!curl -o topic_overview.pdf https://raw.githubusercontent.com/maxschmaltz/Course-LLM-based-Assistants/main/llm-based-assistants/sessions/block2_core/1211_1311/topic_overview.pdf

In the [last session](../0611.ipynb), we created a basic chatbot implemented with LangGraph. The chatbot was built as a graph-like system with the following components:
1. The input receival node. It prompted the user for the input and stored it in the messages for further interaction with the LLM.
2. The router node. It performed the check whether the user wants to exit.
3. The chatbot node. It received the input if the user had not quit, passed it to the LLM, and returned the generation.

Before we begin, let's pull this chatbot, as it will be used as the base class for the further RAG chatbots.

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.rate_limiters import InMemoryRateLimiter

# read system variables
import os
import dotenv

dotenv.load_dotenv()    # that loads the .env file variables into os.environ

In [ ]:
# choose any model, catalogue is available under https://build.nvidia.com/models
MODEL_NAME = "meta/llama-3.3-70b-instruct"

# this rate limiter will ensure we do not exceed the rate limit
# of 40 RPM given by NVIDIA
rate_limiter = InMemoryRateLimiter(
    requests_per_second=35 / 60,  # 35 requests per minute to be sure
    check_every_n_seconds=0.1,  # wake up every 100 ms to check whether allowed to make a request,
    max_bucket_size=7,  # controls the maximum burst size
)

llm = ChatNVIDIA(
    model=MODEL_NAME,
    api_key=os.getenv("NVIDIA_API_KEY"), 
    temperature=0,   # ensure reproducibility,
    rate_limiter=rate_limiter  # bind the rate limiter
)

In [ ]:
from typing import Annotated, List
from typing_extensions import TypedDict
from IPython.display import Image, display
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
class SimpleState(TypedDict):
    # `messages` is a list of messages of any kind. The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[List[BaseMessage], add_messages]
    # Since we didn't define a function to update it, it will be rewritten at each transition
    # with the value you provide
    n_turns: int    # just for demonstration
    language: str   # new, for demonstration

In [ ]:
class Chatbot:

    _graph_path = "./graph.png"
    
    def __init__(self, llm):
        self.llm = llm
        self._build()
        self._display_graph()

    def _build(self):
        # graph builder
        self._graph_builder = StateGraph(SimpleState)
        # add the nodes
        self._graph_builder.add_node("input", self._input_node)
        self._graph_builder.add_node("respond", self._respond_node)
        # define edges
        self._graph_builder.add_edge(START, "input")
        self._graph_builder.add_conditional_edges("input", self._is_quitting_node, {False: "respond", True: END})
        self._graph_builder.add_edge("respond", "input")
        # compile the graph
        self._compile()

    def _compile(self):
        self.chatbot = self._graph_builder.compile()

    def _input_node(self, state: SimpleState) -> dict:
        user_query = input("Your message: ")
        human_message = HumanMessage(content=user_query)
        # add the input to the messages
        return {
            "messages": human_message   # this will append the input to the messages
        }
    
    def _respond_node(self, state: SimpleState) -> dict:
        messages = state["messages"]    # will already contain the user query
        n_turns = state["n_turns"]
        response = self.llm.invoke(messages)
        # add the response to the messages
        return {
            "messages": response,   # this will append the response to the messages
            "n_turns": n_turns + 1  # and this will rewrite the number of turns
        }
    
    def _is_quitting_node(self, state: SimpleState) -> dict:
        # check if the user wants to quit
        user_message = state["messages"][-1].content
        return user_message.lower() == "quit"
    
    def _display_graph(self):
        display(
            Image(
                self.chatbot.get_graph().draw_mermaid_png(
                    output_file_path=self._graph_path
                )
            )
        )

    # add the run method
    def run(self, language=None):
        input = {
            "messages": [
                SystemMessage(
                    content="You are a helpful and honest assistant." # role
                )
            ],
            "n_turns": 0,
            "language": language or "English"    # new
        }
        for event in self.chatbot.stream(input, stream_mode="values"):   #stream_mode="updates"):
            for key, value in event.items():
                print(f"{key}:\t{value}")
            print("\n")

<h2 id="data">1. Data Preprocessing 📕</h2>

As you remember from the lecture, the first step to RAG is data preprocessing. That includes:
1. Loading: load the source (document, website etc.) as a text.
2. Chunking: chunk the loaded text onto smaller pieces.
3. Converting to embeddings: embed the chunks into dense vector for further similarity search.
4. Indexing: put the embeddings into a so-called index -- a special database for efficient storage and search of vectors.

### Loading

We will take a PDF version of the Topic Overview for the last iteration of this course (the current version does not convert nicely). No LLM can know the contents of it, especially some highly specific facts such as dates or key points.

One of ways to load a PDF is to use [`PyPDFLoader`](https://docs.langchain.com/oss/python/integrations/document_loaders/pypdfloader) that load simple textual PDFs and their metadata. In this tutorial, we focus on a simpler variant when there are no multimodal data in the PDF. You can find out more about advanced loading on the PyPDFLoader [LangChain page](https://docs.langchain.com/oss/python/integrations/document_loaders/pypdfloader).

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
file_path = "./topic_overview.pdf"

In [ ]:
loader = PyPDFLoader(file_path)
docs = loader.load()

This function returns a list of `Document` objects, each containing the text of the PDF and its metadata such as title, page, creation date etc.

In [ ]:
docs

In [ ]:
print(docs[0].page_content)

As you can see, the result is not satisfying because the PDF has a more complex structure than just one-paragraph text. To handle it's layout, we could use `UnstructuredLoader`-like OCR engines that will return a `Document` not for the whole page but for a single structure. One of the open-source solutions is [Docling](https://docs.langchain.com/oss/python/integrations/document_loaders/docling).

In [ ]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType

In [ ]:
loader = DoclingLoader(file_path)
docs = loader.load()

Look at how Docling parsed a single structure (left sidebar) into a separate `Document`:

In [ ]:
print(docs[0].page_content)

However, it also parsed every paragraph from the body of the PDF into a separate `Document`, which destroyed the links between the paragraphs of the same sections: 

In [ ]:
print(docs[5].page_content)

To avoid such a fine chunking, we ask the engine to output the whole PDF as a single MD file and then chunk it ourselves.

In [ ]:
loader = DoclingLoader(file_path, export_type=ExportType.MARKDOWN)
docs = loader.load()

In [ ]:
len(docs)

In [ ]:
print(docs[0].page_content)

### Chunking

During RAG, relevant documents are usually retrieved by semantic similarity that is calculated between the search query and each document in the index. However, if we calculate vectors for the entire PDF pages, we risk not to capture any meaning in the embedding because the context is just too long. That is why usually, loaded text is _chunked_ in a RAG application; embeddings for smaller pieces of text are more discriminative, and thus the relevant context may be retrieved more reliably. Furthermore, it ensure process consistency when working documents of varying sizes.

Different approaches to chunking are described in tutorial [Text splitters](https://docs.langchain.com/oss/python/integrations/splitters/index#text-splitters) from LangChain. Even though Docling returned a MD document, we won't be using the dedicated [`MarkdownHeaderTextSplitter`](https://docs.langchain.com/oss/python/integrations/splitters/markdown_header_metadata_splitter); instead, for a more general picture, we choose the [`RecursiveCharacterTextSplitter`](https://docs.langchain.com/oss/python/integrations/splitters/recursive_text_splitter) -- a good option in terms of simplicity-quality ratio for simple cases. This splitter tries to keep text structures (paragraphs, sentences) together and thus maintain text coherence in chunks.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from typing import List

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700, # maximum number of characters in a chunk
    chunk_overlap=250, # number of characters to overlap between chunks
    separators=["\n"]
)

def split_page(doc: Document) -> List[Document]:
    chunks = text_splitter.split_text(doc.page_content)
    return [
        Document(
            page_content=chunk,
            metadata={
                **doc.metadata,
                "chunk_n": i
            },
        ) 
        for i, chunk in enumerate(chunks)
    ]

In [ ]:
chunks = []
for doc in docs:
    chunks += split_page(doc)

print(f"Converted {len(docs)} pages into {len(chunks)} chunks.")

In [ ]:
print(chunks[12].page_content)

### Convert to Embeddings

As discussed, the retrieval usually supported by vector similarity and so the index contains not the actual texts but their vector representations. Vector representations are created by _embedding models_ -- models usually made specifically for this objective by being trained to create more similar vectors for more similar sentences and to push apart dissimilar sentences in the vector space.

We will use the [`nv-embedqa-e5-v5`](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5?snippet_tab=LangChain) model -- a model from NVIDIA pretrained for English QA.

In [ ]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

In [ ]:
EMBEDDING_NAME = "nvidia/nv-embedqa-e5-v5"

embeddings = NVIDIAEmbeddings(
    model=EMBEDDING_NAME, 
    # api_key=os.getenv("NVIDIA_API_KEY")
)

An embedding model receives an input text and returns a dense vector that is believed to capture its semantic properties.

In [ ]:
test_embedding = embeddings.embed_query("retrieval augmented generation")
test_embedding

### Indexing

Now that we have split our data and initialized the embeddings, we can start indexing it. There are a lot of different implementations of indexes, you can take a lot at available options in [Vector stores](https://docs.langchain.com/oss/python/integrations/vectorstores). One of the popular choices is [Qdrant](https://docs.langchain.com/oss/python/integrations/vectorstores/qdrant) that provides a simple data management and can be deployed both locally, on a remote machine, and on the cloud.

Qdrant support persisting your vector storage, i.e. storing it on the working machine, but for simplicity, we will use it in the in-memory mode, so that the storage exists only as long as the notebook does.

In [ ]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from uuid import uuid4

First things first, we need to create a _client_ -- a Qdrant instance that will be the entrypoint for all the actions we do with the data.

In [ ]:
qd_client = QdrantClient(":memory:")    # in-memory Qdrant client

Then, as we use an in-memory client that does not store the index between the notebook sessions, we need to initialize a _collection_. Alternatively, if we were persisting the data, we would perform a check if the collection exists and then either create or load it.

For Qdrant to initialize the structure of the index correctly, we need to provide the dimentionality of the embedding we will be using as well as teh distance metric.

In [ ]:
collection_name = "1211_1311"

qd_client.create_collection(
    collection_name=collection_name,
    # embedding params here
    vectors_config=VectorParams(
        size=len(test_embedding),   # is there a better way?
        distance=Distance.COSINE    # cosine distance
    )
)

Finally, we use a LangChain wrapper to connect to the index to unify the workflow.

In [ ]:
vector_store = QdrantVectorStore(
    client=qd_client,
    collection_name=collection_name,
    embedding=embeddings
)

Now we are ready to add our chunks to the vector storage. As we will be adding the chunks, the index will take care about converting our passages into embeddings.

In order to be able to delete / modify the chunks afterwards, we assign them with unique ids that we generate dynamically.

In [ ]:
ids = [str(uuid4()) for _ in range(len(chunks))]
vector_store.add_documents(
    chunks,
    ids=ids
)

In [ ]:
vector_store.get_by_ids([ids[-1]])

In [ ]:
vector_store.delete([ids[-1]])
vector_store.get_by_ids([ids[-1]])

The index provides the necessary functionality for the query-based retrieval:

In [ ]:
vector_store.similarity_search("retrieval augmented generation", k=3)

<h2 id="prompts">2. Prompting al LangChain 📝</h2>

When you build more complex algorithms, just passing the human query directly might be not enough. Sometimes, you need to give more specific instructions, pre- and append additional stuff to the messages; for example, in (classic) RAG, the retrieved chunks are injected as a context in a dedicated placeholder.

To handle the static instructions more efficiently, you can use `ChatPromptTemplate`. The key idea is simple: in a `ChatPromptTemplate`, you write all the static fragments in plain text and then use placeholders to mark the places where some variable parts will be added. Then, when you receive an input, LangChain fills the placeholders and you receive the desired version of the message with all the placeholders filled automatically.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, MessagesPlaceholder

Before moving to RAG, let us consider a different case to demonstrate the standalone effect of the prompting: CoT prompting.

In [ ]:
input_template_str = """\
The user is asking a question. Please answer it using step-by-step reasoning. \
On each reasoning step, assess whether this reasoning step is good or not, \
on a scale from 1 to 10.

The user question is:

============
{input}
"""

input_template = ChatPromptTemplate.from_template(input_template_str)

Now, even though the user will provide a simple query as usual, the LLM will receive all the additional instructions you wrote. A `ChatPromptTemplate` uses **keys** to fill the placeholders so you should pass it a corresponding `dict`.

In [ ]:
example = input_template.invoke(
    {
        "input": "What is the distance between the Earth and the Moon divided by the number of the planets in the Solar system?"
    }
)

example

In [ ]:
print(example.messages[0].content)

You can also make higher level prompt templates -- that is, with placeholders not for a single message, but for a sequence of messages. To do so, you need to nest `ChatPromptTemplate`s for separate messages and use the `MessagesPlaceholder` for sequences. This approach gives you a universal way to fill the placeholders, be it a separate fragment of a certain message or a whole sequence of messages: all you need is to be careful with the keys, and LangChain will take care of the rest.

In [ ]:
system_template = SystemMessagePromptTemplate.from_template("Answer in the following language: {language}.")

prompt_template = ChatPromptTemplate.from_messages(
    [
        system_template,
        MessagesPlaceholder(variable_name="messages")   # here, you add an entire sequence of messages
    ]
)

Alternative: pass separate messages as pairs of raw strings where the first string describes the role (`"system"`, `"user"`, `"assistant"`) and the second -- the content.

In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer in the following language: {language}."),    # here, you modify a fragment of the system message
        MessagesPlaceholder(variable_name="messages")   # here, you add an entire sequence of messages
    ]
)

In [ ]:
prompt_template.invoke({
    "language": "Spanish",
    "messages": example.to_messages()
})

We can now incorporate this logic into our chatbot.

In [ ]:
class CoTChatbot(Chatbot):
    
    def __init__(self, llm):
        super().__init__(llm)
        # add templates
        self.input_template = input_template
        self.prompt_template = prompt_template

    def _input_node(self, state: SimpleState) -> dict:
        user_query = input("Your message: ")
        if user_query != "quit":
            # invoke the template here
            human_message = self.input_template.invoke(
                {
                    "input": user_query
                }
            ).to_messages()
        else:
            human_message = HumanMessage(content=user_query)
        n_turns = state["n_turns"]
        # add the input to the messages
        return {
            "messages": human_message
        }
    
    def _respond_node(self, state: SimpleState) -> dict:
        # invoke the template here;
        # since the state is already a dictionary, we can just pass it as is
        prompt = self.prompt_template.invoke(state)
        n_turns = state["n_turns"]
        response = self.llm.invoke(prompt)
        # add the response to the messages
        return {
            "messages": response,
            "n_turns": n_turns + 1
        }

    def run(self, language=None):
        # since the system message is now part of the prompt template,
        # we don't need to add it to the input
        input = {
            "messages": [],
            "n_turns": 0,
            "language": language or "English"    # new
        }
        for event in self.chatbot.stream(input, stream_mode="values"):
            if event["messages"]:
                event["messages"][-1].pretty_print()
                print("\n")

In [ ]:
cot_chatbot = CoTChatbot(llm)

In [ ]:
cot_chatbot.run("German")

<h2 id="rag">3. Basic RAG 💉</h2>

Now let's apply our new prompting tool to the basic RAG workflow. For that, we will just retrieve _k_ most relevant documents and them insert them into the prompt as a part of the context.

We will combine the skills we have obtained so far to build a LangGraph agent that receives an input, checks if the user wants to quit, then do the retrieval and generate a context-aware response if not. We will build on the basic version of our first chatbot; to add the RAG functionality, we need to add a retrieval node and modify the generation prompt to inject the retrieved documents.

LangGraph provides a pre-built tool to conveniently create a retriever tool. The retriever tool can be connected to the LLM; in this case, the LLM decides itself whether to use it or not. But for our RAG labs, we separate the tool from the LLM to concentrate on the retrieval and not the agentic aspect of the system (which will be covered in the next week's lab). We also won't generate queries for the retriever for now.

In [ ]:
from langchain.tools.retriever import create_retriever_tool

In [ ]:
# role: restrict it from the parametric knowledge
rag_system_prompt = """\
You are an assistant that has access to a knowledge base. \
You should use the knowledge base to answer the user's questions \
unless you can reliably answer them from your own knowledge.
"""


# this will add the context to the input
context_injection_prompt = """\
The user is asking a question. \
Answer using the following knowledge from the knowledge base:


==========================
{context}
==========================


The user question is:
{input}
"""


# finally, gather the system message, the previous messages,
# and the input with the context
rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", rag_system_prompt),   # system message
        MessagesPlaceholder(variable_name="messages"),  # previous messages
        ("user", context_injection_prompt)  # user message
    ]
)

In [ ]:
class BasicRAGChatbot(Chatbot):

    _graph_path = "./graph_basic_rag.png"
    
    def __init__(self, llm, k=5):
        super().__init__(llm)
        self.rag_prompt = rag_prompt
        self.retriever = vector_store.as_retriever(search_kwargs={"k": k})    # retrieve 5 documents
        self.retriever_tool = create_retriever_tool(    # and this is the tool
            self.retriever,
            "retrieve_internal_data",  # name
            "Search relevant information in internal documents.",   # description
        )

    def _build(self):
        # graph builder
        self._graph_builder = StateGraph(SimpleState)
        # add the nodes
        self._graph_builder.add_node("input", self._input_node)
        self._graph_builder.add_node("retrieve", self._retrieve_node)
        self._graph_builder.add_node("respond", self._respond_node)
        # define edges
        self._graph_builder.add_edge(START, "input")
        # basic rag: no planning, just always retrieve
        self._graph_builder.add_conditional_edges("input", self._is_quitting_node, {False: "retrieve", True: END})
        self._graph_builder.add_edge("retrieve", "respond")
        self._graph_builder.add_edge("respond", "input")
        # compile the graph
        self._compile()
    
    def _retrieve_node(self, state: SimpleState) -> dict:
        # retrieve the context
        user_query = state["messages"][-1].content  # use the last message as the query
        context = self.retriever_tool.invoke({"query": user_query})
        # add the context to the messages
        return {
            "messages": AIMessage(content=context)
        }
    
    def _respond_node(self, state: SimpleState) -> dict:
        # the workflow is designed so that the context is always the last message
        # and the user query is the second to last message
        context = state["messages"].pop(-1).content # don't want the context in the messages
        user_query = state["messages"][-1].content
        prompt = self.rag_prompt.invoke(
            {
                "messages": state["messages"],  # this goes to the message placeholder
                "context": context,  # this goes to the user message
                "input": user_query    # this goes to the user message
            }
        )
        response = self.llm.invoke(prompt)
        # add the response to the messages
        return {
            "messages": response
        }

    def run(self):
        input = {"messages": []}
        for event in self.chatbot.stream(input, stream_mode="values"):
            if event["messages"]:
                event["messages"][-1].pretty_print()
                print("\n")

In [ ]:
basic_rag_chatbot = BasicRAGChatbot(llm)

In [ ]:
basic_rag_chatbot.run()
# What sessions do I have about virtual assistants?
# What is the reading for the third one?

As you can see, it already works pretty well, but as the retrieval goes by the user query directly, the previous context of the conversation is not considered by the _retriever_. To handle that, let's add a node that would reformulate the query taking in consideration the previous interactions.

For that, we need an additional prompt.

In [ ]:
complete_query_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),  # previous messages
        (
			"user",
			"Complete the following user query in the last message to a full question based on the previous messages. "
            "Return only the reformulated user query, without any other text. Do not expand the query "
			"or change its meaning and only fill in the missing information in the last question. "
			"\n\nUser query:\n{query}\n\nReformulated query:"
        )
    ]
)

In [ ]:
class BasicPlusRAGChatbot(BasicRAGChatbot):

    _graph_path = "./graph_basic_plus_rag.png"

    def __init__(self, llm, k=5):
        super().__init__(llm, k)
        self.complete_query_prompt = complete_query_prompt

    def _build(self):
        # graph builder
        self._graph_builder = StateGraph(SimpleState)
        # add the nodes
        self._graph_builder.add_node("input", self._input_node)
        self._graph_builder.add_node("complete_query", self._complete_query_node)
        self._graph_builder.add_node("retrieve", self._retrieve_node)
        self._graph_builder.add_node("respond", self._respond_node)
        # define edges
        self._graph_builder.add_edge(START, "input")
        # basic rag: no planning, just always retrieve
        self._graph_builder.add_conditional_edges("input", self._is_quitting_node, {False: "complete_query", True: END})
        self._graph_builder.add_edge("complete_query", "retrieve")
        self._graph_builder.add_edge("retrieve", "respond")
        self._graph_builder.add_edge("respond", "input")
        # compile the graph
        self._compile()

    def _complete_query_node(self, state: SimpleState) -> dict:
        # since we use the generated query instead of the user query,
        # we want to remove the original user query from the messages
        # and replace it with the generated one, having converted it to a HumanMessage
        if len(state["messages"]) == 1:
            return {}  # no need to reformulate the first query
        user_query = state["messages"].pop(-1)
        prompt = self.complete_query_prompt.invoke({
            **state,    # will take the messages from here
            "query": user_query.content   # and the user query from here
        })
        generated_query = self.llm.invoke(prompt)  
        return {
            "messages": [HumanMessage(content=generated_query.content)] # append the generated query to the messages
        }

In [ ]:
basic_plus_rag_chatbot = BasicPlusRAGChatbot(llm, k=7)

In [ ]:
basic_plus_rag_chatbot.run()
# What sessions do I have about virtual assistants?
# What is the reading for the third one?
# And second?

<h2 id="adv_rag">4. Advanced RAG 😎</h2>

Now we can move to a more complicated implementation. We will now make an iterative RAG chatbot: this chatbot will retrieve contexts iteratively and decide at each step whether the chunks retrieved so far are sufficient to answer the question; the answer is generated only when the retrieved contexts are enough (or when the limit on number of the contexts is reached).

Basically, we have almost everything we need to implement an iterative RAG pipeline. We only need to add three more nodes:
1. A node to (re)generate search queries for the index: now we won't use the user query but specifically generate queries for the index.
2. A decision node, in which the LLM will decide whether the context retrieved so far is enough to proceed to the generation of the response.
3. As a useful addition, we will also add LLM-based filtering of the retrieved documents to filter out the documents that are semantically similar to the query but are not really relevant for answering the question.

We will start by transforming the state to accumulate the contexts gathered so far.

In [ ]:
class AdvancedRAGState(SimpleState): # "messages" is already defined in SimpleRAGState
    queries: List[List[str]]            # this is the list of generated queries, one list per query generation
    docs: List[List[Document]]    # this is the list of retrieved documents, one list per retrieval

To (re)generate the queries, filter the retrieved documents, and decide whether the contexts are supportive enough, we add respective prompt templates.

In [ ]:
generate_query_template = """\
The user is now asking a question. \
Your task is to think of a set of queries that will retrieve all necessary documents \
from the knowledge base to answer the user question. \
Return a comma-separated list of generated queries only (1-3 queries), without any other text.
Imagine you are trying to find a user find information in the internet \
on the topic they are interested in: don't just repeat the same query in different words \
but try to understand what information is required for answering the question and \
what queries would retrieve this information.

Below is a list of previously generated queries, if any. \
The previous queries retrieved documents turned out to be insufficient \
to answer the user's question. Avoid repeating the previous queries \
and explore different aspects of the question.

Previous queries:
{previous_queries}

The user question is:
{input}

Your generated queries:
"""

generate_query_prompt = ChatPromptTemplate.from_messages(
    [
		("system", rag_system_prompt),
        ("user", generate_query_template)
    ]
)

In [ ]:
document_relevant_template = """\
The user is now asking a question. \
For answering the question, your colleague has retrieved the document below. \
Return the degree 1-5 to which the document is related to the user's question, \
where 1 means the document is about something absolutely different, and \
5 means the document is exactly about the topic brought up in the user's question.


===========================
Document:

{document}
===========================


The user question is:
{input}

Rating (1-5):
"""

document_relevant_prompt = ChatPromptTemplate.from_messages(
    [
		("system", rag_system_prompt),
        ("user", document_relevant_template)
    ]
)

In [ ]:
documents_sufficient_template = """\
The user is now asking a question. \
For answering the question, your colleague has retrieved the documents below. \
Return True if the documents are sufficient for answering the question, and False otherwise.


===========================
Documents:

{documents}
===========================


The user question is:
{input}

Verdict (True/False):
"""

documents_sufficient_prompt = ChatPromptTemplate.from_messages(
    [
		("system", rag_system_prompt),
        ("user", documents_sufficient_template)
    ]
)

To reliably parse the generations, we need to add the following structure output schemes:
1. String list for the generated queries.
2. Rating 1-5 for document relevancy.
3. Boolean `True`/`False` for binary documents sufficiency.

In [ ]:
from pydantic import BaseModel, Field

In [ ]:
class ListOutput(BaseModel):
    outputs: List[str] = Field(..., description="List of generated outputs.")

In [ ]:
class Rating1_5(BaseModel):
    rating: int = Field(..., description="Rating from 1 to 5.")

In [ ]:
class YesNoVerdict(BaseModel):
    verdict: bool = Field(..., description="Boolean answer to the given binary question.")

For advanced RAG, we change the LLM and the settings to increase the overall stability.

In [ ]:
# choose any model, catalogue is available under https://build.nvidia.com/models
MODEL_NAME_ADVANCED_RAG = "meta/llama-3.1-405b-instruct"

# this rate limiter will ensure we do not exceed the rate limit
# of 40 RPM given by NVIDIA
rate_limiter_advanced_rag = InMemoryRateLimiter(
    requests_per_second=1 / 6,  # 10 requests per minute to be sure
    check_every_n_seconds=5,  # wake up every 5 seconds to check whether allowed to make a request,
    max_bucket_size=1  # controls the maximum burst size
)

llm_advanced_rag = ChatNVIDIA(
    model=MODEL_NAME_ADVANCED_RAG,
    # api_key=os.getenv("NVIDIA_API_KEY"), 
    temperature=0,   # ensure reproducibility,
    rate_limiter=rate_limiter_advanced_rag  # bind the rate limiter
)

In [ ]:
class IterativeRAGChatbot(BasicPlusRAGChatbot):

    _graph_path = "./graph_iterative_rag.png"

    def __init__(self, llm, k=5, max_generations=3):
        super().__init__(llm, k)
        self.max_generations = max_generations
        self.query_generator = generate_query_prompt | llm.with_structured_output(ListOutput)
        self.document_relevant_grader = document_relevant_prompt | llm.with_structured_output(Rating1_5)
        self.documents_sufficient_grader = documents_sufficient_prompt | llm.with_structured_output(YesNoVerdict)

    def _build(self):
        # graph builder
        self._graph_builder = StateGraph(AdvancedRAGState)
        # add the nodes
        self._graph_builder.add_node("input", self._input_node)
        self._graph_builder.add_node("complete_query", self._complete_query_node)
        self._graph_builder.add_node("generate_query", self._generate_query_node)
        self._graph_builder.add_node("retrieve", self._retrieve_node)
        self._graph_builder.add_node("filter_documents", self._filter_documents_node)
        self._graph_builder.add_node("respond", self._respond_node)
        # define edges
        self._graph_builder.add_edge(START, "input")
        # basic rag: no planning, just always retrieve
        self._graph_builder.add_conditional_edges("input", self._is_quitting_node, {False: "complete_query", True: END})
        self._graph_builder.add_edge("complete_query", "generate_query")
        self._graph_builder.add_edge("generate_query", "retrieve")
        self._graph_builder.add_edge("retrieve", "filter_documents")
        self._graph_builder.add_conditional_edges(
            "filter_documents",
            self._documents_sufficient_node,
            {
                False: "generate_query",
                True: "respond",
                None: END   # max generations reached
            }
        )
        self._graph_builder.add_edge("respond", "input")
        # compile the graph
        self._compile()

    def _generate_query_node(self, state: AdvancedRAGState) -> dict:    
        complete_query = state["messages"][-1].content  # complete user query
        previous_queries = state["queries"][-1] if state["queries"] else []
        generated_queries = self.query_generator.invoke(
            {
                # put previous queries
                "previous_queries": "\n".join(previous_queries) or "NONE",
                "input": complete_query
            }
        )
        return {
            # could have also used `Annotated` here
            "queries": state["queries"] + [generated_queries.outputs]   # List[str]
        }

    # now store the documents in the separate field
    def _retrieve_node(self, state: AdvancedRAGState) -> dict:    
        # retrieve the documents
        latest_queries = state["queries"][-1]  # the last generated queries
        # now use the retriever directly to get a list of documents and not a combined string
        latest_documents = state["docs"][-1] if state["docs"] else []
        newest_documents = []
        for query in latest_queries:
             documents = self.retriever.invoke(query)
             newest_documents.extend([doc for doc in documents if doc not in newest_documents])
        # add the document to the messages
        newest_documents = [doc for doc in newest_documents if doc not in latest_documents]
        return {
            # could have also used `Annotated` here
            "docs": state["docs"] + [newest_documents]
        }
    
    def _filter_documents_node(self, state: AdvancedRAGState) -> dict:
        complete_query = state["messages"][-1].content  # complete user query
        # since the retrieved documents are graded at the same step,
        # we only need to pass the last batch of documents
        latest_documents = state["docs"].pop(-1)  # will be replaced with the filtered ones
        # grade each document separately and only keep the relevant ones
        latest_documents_relevant = []
        for document in latest_documents:
            print("Grading document:\n\n", document.page_content)
            rating = self.document_relevant_grader.invoke(
                {
                    "document": document.page_content,    # this is a Document object
                    "input": complete_query
                }
            )
            print("\n\n")
            print(f"Rating: {rating.rating}")
            print("\n\n=====================\n\n")
            if rating.rating >= 3:    # boolean value according to the Pydantic model
                latest_documents_relevant.append(document)
        return {
            # could have also used `Annotated` here
            "docs": state["docs"] + [latest_documents_relevant]
        }
    
    def _flatten_documents(self, state: AdvancedRAGState) -> List[Document]:
        all_documents = state["docs"]
        return [document for sublist in all_documents for document in sublist]

    def _documents_sufficient_node(self, state: AdvancedRAGState) -> dict:
        complete_query = state["messages"][-1].content  # complete user query
        documents = self._flatten_documents(state)
        if not documents:
            return False
        documents_str = "\n\n".join([document.page_content for document in documents])
        print("Deciding whether the documents are sufficient")
        verdict = self.documents_sufficient_grader.invoke(
                {
                "documents": documents_str,    # this is a Document object
                "input": complete_query
            }
        )
        print("\n\n")
        print(f"Verdict: {verdict.verdict}")
        print("\n\n=====================\n\n")
        if not verdict.verdict and len(documents) == self.max_generations:
            return  # will route to END
        return verdict.verdict
    
    def _respond_node(self, state: AdvancedRAGState) -> dict:
        documents = self._flatten_documents(state)
        documents_str = "\n\n".join([document.page_content for document in documents])
        complete_query = state["messages"][-1].content
        prompt = self.rag_prompt.invoke(
            {
                "messages": state["messages"],  # this goes to the message placeholder
                "context": documents_str,  # this goes to the user message
                "input": complete_query    # this goes to the user message
            }
        )
        response = self.llm.invoke(prompt)
        # add the response to the messages
        return {
            "messages": response
        }
    
    def run(self):
        input = {"messages": [], "docs": [], "queries": []}
        for event in self.chatbot.stream(input, stream_mode="updates"):
            for key, value in event.items():
                if value and value.get("messages"):
                    # print(key, end="\n\n")
                    if not isinstance(value["messages"], list):
                        value["messages"] = [value["messages"]]
                    for message in  value["messages"]:
                        message.pretty_print()
                    print("\n\n")
                elif value and value.get("queries"):
                    print(f"\n\nGenerated queries: {', '.join(value['queries'][-1])}\n\n")

In [ ]:
# make very small k to ensure one retrieval is not enough
iterative_rag_chatbot = IterativeRAGChatbot(llm_advanced_rag, k=1)

In [ ]:
iterative_rag_chatbot.run()
# Summarize the topics of all the pitches in two sentences.